# 데이터 전처리

## 1. 표준화 또는 평균 제거와 분산 스케일링

표준화는 많은 머신러닝 추정기에서 보통 필요로 한다. 

전처리 모듈은 array같은 데이터셋에 연산을 빠르게 수행하는 StandardScaler 유틸리티 클래스를 제공한다.

In [2]:
from sklearn import preprocessing
import numpy as np
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])

In [3]:
scaler = preprocessing.StandardScaler().fit(X_train)
scaler

StandardScaler()

In [4]:
scaler.mean_

array([1.        , 0.        , 0.33333333])

In [5]:
scaler.scale_

array([0.81649658, 0.81649658, 1.24721913])

In [6]:
X_scaled = scaler.transform(X_train)
X_scaled

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

In [7]:
X_scaled.mean(axis=0)

array([0., 0., 0.])

In [8]:
X_scaled.std(axis=0)

array([1., 1., 1.])

이 클래스는 Transformer API를 구현하여 학습 세트에서 평균과 표준 편차를 계산하여 나중에 테스트 세트에서 동일한 변환을 다시 적용할 수 있도록 한다.

파이프라인 초기 단계에서 사용하기 적합니다.

In [9]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

X, y = make_classification(random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe.fit(X_train, y_train)  # apply scaling on training data

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [10]:
pipe.score(X_test, y_test)  # apply scaling on testing data, without leaking training data.

0.96

## 1.1 특징 값을 범위 내로 스케일링하기

특정 최소 ~ 최대값 범위 내로 특징 값을 스케일링하는 표준화 방법이 일반적이다.

이 스케일링을 사용하는 이유는 특징의 매우 작은 표준편차에 대해 데이터에서 0개의 항목을 보존한다.

In [11]:
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])

min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_train_minmax

array([[0.5       , 0.        , 1.        ],
       [1.        , 0.5       , 0.33333333],
       [0.        , 1.        , 0.        ]])

동일한 변환기의 인스턴스는 학습과정에 없었던 새로운 테스트 데이터에도 적용될 수 있다.


In [12]:
X_test = np.array([[-3., -1.,  4.]])
X_test_minmax = min_max_scaler.transform(X_test)
X_test_minmax

array([[-1.5       ,  0.        ,  1.66666667]])

In [13]:
min_max_scaler.scale_

array([0.5       , 0.5       , 0.33333333])

In [14]:
min_max_scaler.min_

array([0.        , 0.5       , 0.33333333])

MaxAbsScaler는 각 특성의 가장 큰 최댓값으로 나눠 학습 데이터가 [-1,1] 범위 내에 위치하도록 스케일을 조정한다.

이는 이미 0을 중심으로 정렬된 데이터나 희소한 데이터에 적합하다.

In [15]:
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])
max_abs_scaler = preprocessing.MaxAbsScaler()
X_train_maxabs = max_abs_scaler.fit_transform(X_train)
X_train_maxabs

array([[ 0.5, -1. ,  1. ],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  1. , -0.5]])

In [16]:
X_test = np.array([[ -3., -1.,  4.]])
X_test_maxabs = max_abs_scaler.transform(X_test)
X_test_maxabs

array([[-1.5, -1. ,  2. ]])

In [17]:
max_abs_scaler.scale_

array([2., 1., 2.])

MinMaxScaler와 MaxAbsScaler는 입력 데이터의 특성 값을 고정된 범위로 저정하여 데이터의 스케일을 조정하는데 이 두 스케일러는 모두 수치형 데이터에서 잘 작동한다.

데이터가 이미 중심화되어 있거나 희소 데이터일 경우, MaxAbsScaler를 사용하여 스케일링하는 것이 좋지만, StandardScaler는 Wirh_mean=False를 지정하여 constructor에 전달하면 scipysparse희소행렬도 입력으로 사용할 수 있다.

데이터에 이상치가 많은 경우, 데이터의 평균과 분산을 사용하여 스케일링하는 것이 잘 작동하지 않을 수 있다. 이 경우 RobustScaler를 사용하여 중심화와 범위의 보다 견고한 추정치를 사용할 수 있다.

특성간 선형 상관관계를 제거하기 위해 PCA를 사용하여 데이터를 화이트닝할 수 있습니다. 이를 통해 다운스트림 모델에서 특성 간의 선형 독립성을 가정할 수 있다.

## 2. 비선형 변환

스케일링이 아닌 다른 종류의 변환인 quantile transforms power transforms를 한다.

두개 모두 단조변환을 기반으로 하므로 각 특성의 값의 순위를 보존한다.

power transform는 모든 분포에서 데이터를 가우시안 분포에 가깝게 매핑하도록 설계되어 데이터의 정규성을 향상시키는데 사용할 수 있다.

In [18]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
quantile_transformer = preprocessing.QuantileTransformer(random_state=0)
X_train_trans = quantile_transformer.fit_transform(X_train)
X_test_trans = quantile_transformer.transform(X_test)
np.percentile(X_train[:, 0], [0, 25, 50, 75, 100]) 

C:\Users\kimjeongyeon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_data.py:2627: UserWarning: n_quantiles (1000) is greater than the total number of samples (112). n_quantiles is set to n_samples.
  warnings.warn(


array([4.3, 5.1, 5.8, 6.5, 7.9])

In [19]:
np.percentile(X_train_trans[:, 0], [0, 25, 50, 75, 100])

array([0.        , 0.23873874, 0.50900901, 0.74324324, 1.        ])

In [20]:
np.percentile(X_test[:, 0], [0, 25, 50, 75, 100])

array([4.4  , 5.125, 5.75 , 6.175, 7.3  ])

In [21]:
np.percentile(X_test_trans[:, 0], [0, 25, 50, 75, 100])

array([0.01351351, 0.25      , 0.47747748, 0.60472973, 0.94144144])

모델링 시나리오에서 데이터셋의 특성이 정규성을 가질 수 있도록 하는 것이 바람직한 경우에 대해 다룬다.

Power transform은 모든 분포에서 데이터를 가우시안 분포를 가깝게 매핑하도록 설계된 매개변수적, 단조 변환이다.

이를 통해 분산을 안정화하고 skewness를 최소화할 수 있다.

box-cox 변환은 양수 데이터에만 적용할 수 있다.

In [22]:
pt = preprocessing.PowerTransformer(method='box-cox', standardize=False)
X_lognormal = np.random.RandomState(616).lognormal(size=(3, 3))
X_lognormal

array([[1.28331718, 1.18092228, 0.84160269],
       [0.94293279, 1.60960836, 0.3879099 ],
       [1.35235668, 0.21715673, 1.09977091]])

In [23]:
pt.fit_transform(X_lognormal)

array([[ 0.49024349,  0.17881995, -0.1563781 ],
       [-0.05102892,  0.58863195, -0.57612414],
       [ 0.69420009, -0.84857822,  0.10051454]])

power transforms는 가우시안 같은 결과를 도출해네지만 비효율적이다.

In [24]:
quantile_transformer = preprocessing.QuantileTransformer(
    output_distribution='normal', random_state=0)
X_trans = quantile_transformer.fit_transform(X)
quantile_transformer.quantiles_

C:\Users\kimjeongyeon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_data.py:2627: UserWarning: n_quantiles (1000) is greater than the total number of samples (150). n_quantiles is set to n_samples.
  warnings.warn(


array([[4.3, 2. , 1. , 0.1],
       [4.4, 2.2, 1.1, 0.1],
       [4.4, 2.2, 1.2, 0.1],
       [4.4, 2.2, 1.2, 0.1],
       [4.5, 2.3, 1.3, 0.1],
       [4.6, 2.3, 1.3, 0.2],
       [4.6, 2.3, 1.3, 0.2],
       [4.6, 2.3, 1.3, 0.2],
       [4.6, 2.4, 1.3, 0.2],
       [4.7, 2.4, 1.3, 0.2],
       [4.7, 2.4, 1.3, 0.2],
       [4.8, 2.5, 1.4, 0.2],
       [4.8, 2.5, 1.4, 0.2],
       [4.8, 2.5, 1.4, 0.2],
       [4.8, 2.5, 1.4, 0.2],
       [4.8, 2.5, 1.4, 0.2],
       [4.9, 2.5, 1.4, 0.2],
       [4.9, 2.5, 1.4, 0.2],
       [4.9, 2.5, 1.4, 0.2],
       [4.9, 2.6, 1.4, 0.2],
       [4.9, 2.6, 1.4, 0.2],
       [4.9, 2.6, 1.4, 0.2],
       [5. , 2.6, 1.4, 0.2],
       [5. , 2.6, 1.4, 0.2],
       [5. , 2.7, 1.5, 0.2],
       [5. , 2.7, 1.5, 0.2],
       [5. , 2.7, 1.5, 0.2],
       [5. , 2.7, 1.5, 0.2],
       [5. , 2.7, 1.5, 0.2],
       [5. , 2.7, 1.5, 0.2],
       [5. , 2.7, 1.5, 0.2],
       [5. , 2.7, 1.5, 0.2],
       [5.1, 2.7, 1.5, 0.2],
       [5.1, 2.8, 1.5, 0.2],
       [5.1, 2

## 3.정규화

정규화는 각 샘플을 단위 벡터norm으로 스케일링하는 과정이다. 

이 과정은 이차형식을 사용할 계획이라면 유용할 것이다.

이 가정은 텍스트 분류 및 클러스터링 컨텍스트에서 자주 사용되는 벡터 공간모델의 기반을 두고 있다.

In [25]:
X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]
X_normalized = preprocessing.normalize(X, norm='l2')
X_normalized

array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])


전처리 모듈은 Normalizer라는 유틸리티 클래스를 제공하는데 transformer API를 사용해 같은 과정을 구현한다.

이 클래스는 파이프라인 초기단계에서 사용하기 적합하다.

In [26]:
normalizer = preprocessing.Normalizer().fit(X)  # fit does nothing
normalizer

Normalizer()

In [27]:
normalizer.transform(X)

array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

In [28]:
normalizer.transform([[-1.,  1., 0.]])

array([[-0.70710678,  0.70710678,  0.        ]])

## 4. 범주형 특성 인코딩

종종 features은 수치형이 아니라 범주형 데이터로 주어질 수 있다.

이러한 범주형 데이터를 정수형의 수치형 코드로 변환하기 위해 순서형 인코더를 사용할 수 있다.

In [29]:
enc = preprocessing.OrdinalEncoder()
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)

OrdinalEncoder()

In [30]:
enc.transform([['female', 'from US', 'uses Safari']])

array([[0., 1., 1.]])

그러나 이러한 정수 표현은 지속적으로 입력이 주어지는 scikit learn 추정기에 적합하지 않다.

범주를 순서가 지정된 것으로 해석할 수 있는 오해를 불러일으킬 수도 있다.

np.nan 결측치도 통과시켜버린다.

In [31]:
enc = preprocessing.OrdinalEncoder()
X = [['male'], ['female'], [np.nan], ['female']]
enc.fit_transform(X)

array([[ 1.],
       [ 0.],
       [nan],
       [ 0.]])

OrdinalEncoder는 파이프라인을 생성할 필요 없이 SimpleImputer를 사용하여 누락된 값을 인코딩하는 매개 변수 encoded_missing_value를 제공한다.

In [32]:
enc = preprocessing.OrdinalEncoder(encoded_missing_value=-1)
X = [['male'], ['female'], [np.nan], ['female']]
enc.fit_transform(X)

array([[ 1.],
       [ 0.],
       [-1.],
       [ 0.]])

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
enc = Pipeline(steps=[
    ("encoder", preprocessing.OrdinalEncoder()),
    ("imputer", SimpleImputer(strategy="constant", fill_value=-1)),
])
enc.fit_transform(X)

array([[ 1.],
       [ 0.],
       [-1.],
       [ 0.]])

범주형 데이터는 OneHotEncode라는 방식으로 사용할 수도 있다.
해당하는 값은 1을 주고 나머지는 0을 넣는다.

In [34]:
enc = preprocessing.OneHotEncoder()
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)

OneHotEncoder()

In [35]:
enc.transform([['female', 'from US', 'uses Safari'],
               ['male', 'from Europe', 'uses Safari']]).toarray()

array([[1., 0., 0., 1., 0., 1.],
       [0., 1., 1., 0., 0., 1.]])

In [36]:
enc.categories_


[array(['female', 'male'], dtype=object),
 array(['from Europe', 'from US'], dtype=object),
 array(['uses Firefox', 'uses Safari'], dtype=object)]

In [37]:
genders = ['female', 'male']
locations = ['from Africa', 'from Asia', 'from Europe', 'from US']
browsers = ['uses Chrome', 'uses Firefox', 'uses IE', 'uses Safari']
enc = preprocessing.OneHotEncoder(categories=[genders, locations, browsers])
# Note that for there are missing categorical values for the 2nd and 3rd
# feature
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)

OneHotEncoder(categories=[['female', 'male'],
                          ['from Africa', 'from Asia', 'from Europe',
                           'from US'],
                          ['uses Chrome', 'uses Firefox', 'uses IE',
                           'uses Safari']])

In [38]:
enc.transform([['female', 'from Asia', 'uses Chrome']]).toarray()

array([[1., 0., 0., 1., 0., 0., 1., 0., 0., 0.]])

학습 데이터에 범주형 변수가 누락될 가능성이 있다면, 위에서 수동으로 범주를 설정하는 대신에 handle_unknown='infrequent_if_exist'를 지정하는 것이 더 나을 수도 있다.

알 수 없는 범주가 발견되면 오류가 발생하지 않지만 해당 특성의 OneHotEncoding 열은 모두 0 이거나 활성화되면 드물게 나타나는 범주로 간주된다.

In [39]:
enc = preprocessing.OneHotEncoder(handle_unknown='infrequent_if_exist')
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)

OneHotEncoder(handle_unknown='infrequent_if_exist')

In [40]:
enc.transform([['female', 'from Asia', 'uses Chrome']]).toarray()

array([[1., 0., 0., 0., 0., 0.]])

drop 매개 변수를 사용하여 n_categories 열 대신 n_categories-1 열로 각 열을 인코딩하는 것도 가능하다.

이러한 기능은 예를 들어, 공선성이 발생할 경우 비정규화 회귀 (LinearRegression)를 사용할 때 유용하다.

In [41]:
X = [['male', 'from US', 'uses Safari'],
     ['female', 'from Europe', 'uses Firefox']]
drop_enc = preprocessing.OneHotEncoder(drop='first').fit(X)
drop_enc.categories_

[array(['female', 'male'], dtype=object),
 array(['from Europe', 'from US'], dtype=object),
 array(['uses Firefox', 'uses Safari'], dtype=object)]

In [42]:
drop_enc.transform(X).toarray()

array([[1., 1., 1.],
       [0., 0., 0.]])

In [43]:
X = [['male', 'US', 'Safari'],
     ['female', 'Europe', 'Firefox'],
     ['female', 'Asia', 'Chrome']]
drop_enc = preprocessing.OneHotEncoder(drop='if_binary').fit(X)
drop_enc.categories_

[array(['female', 'male'], dtype=object),
 array(['Asia', 'Europe', 'US'], dtype=object),
 array(['Chrome', 'Firefox', 'Safari'], dtype=object)]

In [44]:
drop_enc.transform(X).toarray()

array([[1., 0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0., 1., 0.],
       [0., 1., 0., 0., 1., 0., 0.]])

변환된 x에서 첫번째 열은 "남성 여성" 범주가 있는 feature의 인코딩이며 나머지 6개의 열은 각 3개의범주가 있는 2개의 features의 인코딩이다.

In [45]:
drop_enc = preprocessing.OneHotEncoder(drop='first',
                                       handle_unknown='ignore').fit(X)
X_test = [['unknown', 'America', 'IE']]
drop_enc.transform(X_test).toarray()

C:\Users\kimjeongyeon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [0, 1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


array([[0., 0., 0., 0., 0.]])

모든 x_test에 있는 범주는 변환되는 동안 unknown상태이고 0으로 매핑될것이다

In [46]:
drop_enc = preprocessing.OneHotEncoder(drop='if_binary', sparse_output=False,
                                       handle_unknown='ignore').fit(X)
X_test = [['unknown', 'America', 'IE']]
X_trans = drop_enc.transform(X_test)
X_trans

C:\Users\kimjeongyeon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [0, 1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


array([[0., 0., 0., 0., 0., 0., 0.]])

In [47]:
drop_enc.inverse_transform(X_trans)

array([['female', None, None]], dtype=object)

원핫인코더는 결측치가 있는 범주 features를 도와준다.

In [48]:
X = [['male', 'Safari'],
     ['female', None],
     [np.nan, 'Firefox']]
enc = preprocessing.OneHotEncoder(handle_unknown='error').fit(X)
enc.categories_

[array(['female', 'male', nan], dtype=object),
 array(['Firefox', 'Safari', None], dtype=object)]

In [49]:
enc.transform(X).toarray()

array([[0., 1., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0.]])

만약 feature이 np.nan, None을 모두 포함하고 있다면, 범주가 분리되 수도 있다.

In [50]:
X = [['Safari'], [None], [np.nan], ['Firefox']]
enc = preprocessing.OneHotEncoder(handle_unknown='error').fit(X)
enc.categories_

[array(['Firefox', 'Safari', None, nan], dtype=object)]

In [51]:
enc.transform(X).toarray()

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]])

드물게 나타나는 범주에 대해 원핫인코더는 하나의 출력으로 집계하는 기능을 지원한다.

예제에서는 dog, snake는 드물게 나타나는 범주로 간주된다.

In [52]:
X = np.array([['dog'] * 5 + ['cat'] * 20 + ['rabbit'] * 10 +
              ['snake'] * 3], dtype=object).T
enc = preprocessing.OneHotEncoder(min_frequency=6, sparse_output=False).fit(X)
enc.infrequent_categories_

[array(['dog', 'snake'], dtype=object)]

In [53]:
enc.transform(np.array([['dog'], ['cat'], ['rabbit'], ['snake']]))

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

infrequent_if_exist을 handle_unknown 으로 둠으로써, 결측 범주는 드문 것으로 간주된다.

In [54]:
enc = preprocessing.OneHotEncoder(
   handle_unknown='infrequent_if_exist', sparse_output=False, min_frequency=6)
enc = enc.fit(X)
enc.transform(np.array([['dragon']]))

array([[0., 0., 1.]])

In [55]:
enc.get_feature_names_out()


array(['x0_cat', 'x0_rabbit', 'x0_infrequent_sklearn'], dtype=object)

드물게 나타나는 범주를 돕지 않았거나 그런 범주가 없는 경우 해당 원핫인코딩 열은 모두 0이 된다.

만약 드물게 나타나는 범주가 있는 경우, 알 수 없는 범주는 드물게 나타나는 범주로 간ㄴ주된다.

max_categories를 사용하여 드물게 나타나는 범주를 구성할 수 있다.

In [56]:
enc = preprocessing.OneHotEncoder(max_categories=2, sparse_output=False)
enc = enc.fit(X)
enc.transform([['dog'], ['cat'], ['rabbit'], ['snake']])

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [57]:
enc = preprocessing.OneHotEncoder(min_frequency=4, max_categories=3, sparse_output=False)
enc = enc.fit(X)
enc.transform([['dog'], ['cat'], ['rabbit'], ['snake']])


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

max_categories 기준으로 기준 좌표에서 드물게 나타나는 범주가 같은 경우 lexicon 순서를 기반으로 처음것이 선택된다.

다음 예제에서는 b, c가 드물게 나타나는 범주로 처리될 것이다.

In [58]:
X = np.asarray([["a"] * 20 + ["b"] * 10 + ["c"] * 10 + ["d"] * 10], dtype=object).T
enc = preprocessing.OneHotEncoder(max_categories=3).fit(X)
enc.infrequent_categories_


[array(['b', 'c'], dtype=object)]

## 5. 이산화

연속적인 형상을 이산값으로 분할하는 방법을 제공한다.

이산화는 연속성의 데이터세트를 필요한 속성만 있는 데이터 세트로 변환할 수 있기 때문에 연속성이 있는 특정 데이터 세트는 이산화를 해주면 좋다.

원핫인코드된 이산화된 특징들은 모델을 좀더 표현력있게 만들어준다.

이산화로 전처리된 것은 선형모델에 비선형성을 도입할 수 있다.

In [73]:
X = np.array([[ -3., 5., 15 ],
              [  0., 6., 14 ],
              [  6., 3., 11 ]])
est = preprocessing.KBinsDiscretizer(n_bins=[3, 2, 2], encode='ordinal').fit(X)

기본적으로 출력은 원핫인코딩으로 희소 행렬로 인코딩한다.

각  특성에 대해 fit 하는 동안 bin edge가 계산되며, 이와 함께 bin의 수가 결정되어 각 특성의 구간을 정의한다.

In [60]:
est.transform(X)

array([[0., 1., 1.],
       [1., 1., 1.],
       [2., 0., 0.]])

이 결과 데이터 세트에는 파이프라인에서 추가로 사용할 수 있는 순서형 속성이 포함되어 있다.

이산화는 연속 데이터에 대한 히스토그램을 구성하는 것과 유사하다.

그러나 히스토그램은 특정 bin에 속하는 특징들을 계산하는 데 초점을 맞추고 이산화는 이러한 bin에 특징 값을 할당하는 데 초점을 맞춘다.

'uniform'전략은 일정한 너비의 bin을 이용한다. 

'quantile'전략은  분위수 값을 사용하여 동일하게 채워진 bin 들을 갖는다.

이산화 전략을 정의하는 호출기를 Function Transformer에 전달하여 사용자 정의 빈을 지정할 수 있다.

In [74]:
import pandas as pd
import numpy as np
bins = [0, 1, 13, 20, 60, np.inf]
labels = ['infant', 'kid', 'teen', 'adult', 'senior citizen']
transformer = preprocessing.FunctionTransformer(
    pd.cut, kw_args={'bins': bins, 'labels': labels, 'retbins': False}
)
X = np.array([0.2, 2, 15, 25, 97])
transformer.fit_transform(X)

['infant', 'kid', 'teen', 'adult', 'senior citizen']
Categories (5, object): ['infant' < 'kid' < 'teen' < 'adult' < 'senior citizen']

### 특성 이진화

특성 이진화는 숫자 특성을 임계값으로 나누어 부울 값으로 변환하는 과정이다.

이것은 수치형 데이터를 이진 데이터로 변환하는 과정(downstream probabilistic estimators)이 다변수 베르누이 분포를 따른다는 가정을 만들 때 유용하다.

또한 텍스트 처리 커뮤니티에서 이진 특성 값을 사용하는 것이 일반적인데 아마도 확률적 추론을 단순화하기 위한 것일 것이다.

Normalizer와 마찬가지로 유틸리티 클래스인 Binarizer는 파이프라인 초기단계에서 사용하는 것이 좋다.

fit 메서드는 각 샘플을 다른 샘플과 독립적으로 처리하기 때문에 아무것도 하지 않는다.

In [76]:
X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]

binarizer = preprocessing.Binarizer().fit(X)  # fit does nothing
binarizer

Binarizer()

In [77]:
binarizer.transform(X)

array([[1., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [78]:
binarizer = preprocessing.Binarizer(threshold=1.1)
binarizer.transform(X)

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 0.]])

Normalizer 클래스에 대해서는 transformer API가 필요하지 않을 때 사용할 수 있는 동반 함수 binarize가 제공된다.

희소 입력의 경우 데이터 압축 희소 행렬 표현으로 변환된다.

## 6.결측값 대치

많은 실제 데이터셋에는 누락된 값이 포함된다.

이러한 값들은 공백, Nan 또는 다른 placeholder로 인코딩된다.

그러나 scikit learn은 이런 누락된 데이터 셋과 호환되지 않는다.

누락된 값을 포함하는 전체 행,열을 삭제하는 방법을 사용할 수도 있고 누락된 값을 대체하는 방법도 있다.

## 7. 다항식 특성 생성

입력 데이터의 비선형 특성을 고려하여 모델의 목잡성을 추가하는 것이 유용한 경우가 많다.

우리는 두가지 다항식을 기반으로 한 두가지 가능성을 보여준다.(1. 순수 다항식, 2. 분할다항식)

In [80]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
X = np.arange(6).reshape(3, 2)
X

array([[0, 1],
       [2, 3],
       [4, 5]])

In [81]:
poly = PolynomialFeatures(2)
poly.fit_transform(X)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

In [82]:
X = np.arange(9).reshape(3, 3)
X

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [83]:
poly = PolynomialFeatures(degree=3, interaction_only=True)
poly.fit_transform(X)

array([[  1.,   0.,   1.,   2.,   0.,   0.,   2.,   0.],
       [  1.,   3.,   4.,   5.,  12.,  15.,  20.,  60.],
       [  1.,   6.,   7.,   8.,  42.,  48.,  56., 336.]])

비선형 항을 추가하는 다른 방법은 Spline Transformer를 사용하는 것이다.

B-스플라인은 고정된 낮은 차수(3)을 유지하고 knot의 수를 절약적으로 조정하는 경우 매우 유연하고 견고하다.

In [84]:
import numpy as np
from sklearn.preprocessing import SplineTransformer
X = np.arange(5).reshape(5, 1)
X

array([[0],
       [1],
       [2],
       [3],
       [4]])

In [85]:
spline = SplineTransformer(degree=2, n_knots=3)
spline.fit_transform(X)

array([[0.5  , 0.5  , 0.   , 0.   ],
       [0.125, 0.75 , 0.125, 0.   ],
       [0.   , 0.5  , 0.5  , 0.   ],
       [0.   , 0.125, 0.75 , 0.125],
       [0.   , 0.   , 0.5  , 0.5  ]])

## 8.사용자 지정 변환기

기존의 파이썬 함수를 변환하여 데이터 정리 또는 처리를 지원하는 변환기로 만들고 싶을 때가 종종 있다.

Function Transformer를 사용하여 임의의 함수에서 변환기를 구현할 수 있다.

In [87]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer
transformer = FunctionTransformer(np.log1p, validate=True)
X = np.array([[0, 1], [2, 3]])
# Since FunctionTransformer is no-op during fit, we can call transform directly
transformer.transform(X)

array([[0.        , 0.69314718],
       [1.09861229, 1.38629436]])

In [88]:
import warnings
warnings.filterwarnings("error", message=".*check_inverse*.",
                        category=UserWarning, append=False)


# 대략적인 정리